In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
dir_path = './data/'

In [59]:
def load_books_data(file_path):
    books_df = pd.read_excel(file_path, engine='openpyxl')
    return books_df

def create_tfidf_matrix(books_df):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(books_df['description'])
    return tfidf_matrix

def calculate_similarity_matrix(tfidf_matrix):
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return similarity_matrix

def recommend_books(book_isbn, books_df, similarity_matrix, top_n=5):
    indices = pd.Series(books_df.index, index=books_df['isbn'])
    book_index = indices[book_isbn]
    
    similarity_scores = list(enumerate(similarity_matrix[book_index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_similar_books_indices = [i[0] for i in similarity_scores[1:top_n+1]]

    return books_df.iloc[top_similar_books_indices]

if __name__ == '__main__':
    books_data_path = dir_path+'aladin_20230515_final.xlsx'
    books_df = load_books_data(books_data_path)
    
    user_isbn = '9791192512259'
    user_descrip = '빵순이, 빵돌이들의 고요한 삼시세빵 일본 빵지순례 바이블. 일본 현지에서만 맛볼 수 있는 거의 모든 빵들을 모았다. 일본 전역 158개 빵집 또는 빵 제조업체에서 만드는 264종의 빵이다. 빵마다 기원과 성분, 맛의 특징, 소비자의 반응을, 빵집마다 역사와 개성, 주요 메뉴를 명쾌하고 재치 있게 소개했다.'
    user_dic = {'isbn': [user_isbn], 'description': [user_descrip]}
    user_df = pd.DataFrame(user_dic)
    
    recommend_books_df = pd.concat([books_df[['isbn', 'description']], user_df], ignore_index=True)
    recommend_books_df.drop_duplicates(['isbn'], inplace=True ,ignore_index=True)
    
    tfidf_matrix = create_tfidf_matrix(recommend_books_df)
    similarity_matrix = calculate_similarity_matrix(tfidf_matrix)
    
    recommendations = recommend_books(user_isbn, recommend_books_df, similarity_matrix)
    recommendations = books_df.loc[recommendations.index]

In [61]:
recommendations

,isbn,title,description,author,cover
1052,9791192512273,일본 현지 빵 대백과,"빵순이, 빵돌이들의 고요한 삼시세빵 일본 빵지순례 바이블. 일본 현지에서만 맛볼 수...","타쓰미출판 편집부 (지은이), 수키 (옮긴이)",https://image.aladin.co.kr/product/31653/78/co...
1569,9791159057731,안티-재팬 - 탈식민 동아시아의 감정의 정치학,"제2차 세계대전이 끝난 후 일본 제국은 빠르게 해체되었지만, 일본 제국의 착취에 대...","리오 T. S. 칭 (지은이), 유정완 (옮긴이)",https://image.aladin.co.kr/product/31612/30/co...
1775,9791138477987,항구의 니쿠코짱!,"제152회 나오키상, 일본 서점대상 2위를 수상한 일본 대표 여성 작가 니시 가나코...","니시 가나코 (지은이), 이소담 (옮긴이)",https://image.aladin.co.kr/product/31531/11/co...
1787,9791191455755,일본 도자기 여행 : 규슈의 8대 조선 가마 - 개정증보판,"도자기 관련 다수의 저서를 펴낸, 세계 도자문화사 전문가 조용준이 『일본 도자기 여...",조용준 (지은이),https://image.aladin.co.kr/product/31523/47/co...
277,9791191925111,발레 자세 교정 핸드북 - 바른 자세로 운동 효과를 높이는 비결,발레리나 수천 명의 자세를 분석한 일본 최고의 무용수 치료사가 가장 정확한 발레 자...,"시마다 사토시 (지은이), 김지혜 (옮긴이)",https://image.aladin.co.kr/product/31614/76/co...
